In [1]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler

In [2]:
iris=load_iris()
X=pd.DataFrame(iris.data,columns=iris.feature_names)

In [3]:
Scaler=StandardScaler()
X_Scaled=Scaler.fit_transform(X)

In [4]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
kmeans=KMeans(n_clusters=3,random_state=42)
kmeans_labels=kmeans.fit_predict(X_Scaled)

E:\Anilesh\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
E:\Anilesh\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


In [5]:
kmeans_silhouette=silhouette_score(X_Scaled, kmeans_labels)
print(f'K-Means Silhouette Score: {kmeans_silhouette}')

K-Means Silhouette Score: 0.45994823920518635


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
X_tensor=torch.tensor(X_Scaled, dtype=torch.float32)

In [7]:
data_loader=DataLoader(TensorDataset(X_tensor),batch_size=16,shuffle=True)

In [8]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder=nn.Sequential(
            nn.Linear(4,2),
            nn.ReLU()
        )
        self.decoder=nn.Sequential(
            nn.Linear(2,4),
            nn.ReLU()
        )
    def forward(self,x):
        encoded=self.encoder(x)
        decoded=self.decoder(encoded)
        return decoded

In [9]:
autoencoder=Autoencoder()
criterion=nn.MSELoss()
optimizer=optim.Adam(autoencoder.parameters(),lr=0.01)

In [10]:
num_epochs=100
for epoch in range(num_epochs):
    for data in data_loader:
        inputs, =data
        optimizer.zero_grad()
        outputs=autoencoder(inputs)
        loss=criterion(outputs, inputs)
        loss.backward()
        optimizer.step()

In [11]:
with torch.no_grad():
    encoded_data=autoencoder.encoder(X_tensor).numpy()

In [12]:
encoded_kmeans=KMeans(n_clusters=3,random_state=42)
encoded_kmeans_labels=encoded_kmeans.fit_predict(encoded_data)

E:\Anilesh\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
E:\Anilesh\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


In [13]:
encoded_silhouette = silhouette_score(X_Scaled, encoded_kmeans_labels)
print(f'Autoencoder + kmeans silhouette score : {encoded_silhouette}')

Autoencoder + kmeans silhouette score : 0.4048068718871258
